# 1) Заглавие на тема: **Чат бот**

Изготвил:

*Божидар Топчийски*

*Анна-Мария Атанасова*

# 2) Кратко описание:
Взаимодействие с модел и използването му за генериране на разговор.

# 3) Описание на използваните технологии
*   Модел: Llama-2 7B Chat
*   Графичен юзър интерфейс: Gradio ChatInterface

# 4) Описание на техническата разработка

1. **Инсталиране** на библиотеки
2. **Предпоставки** - достъп до модела Llama-2 7B в Hugging Face
3. **Зареждане на модел и токенизатор** - извличането им за настоящата сесия
4. **Създаване на Llama канал** - подготовка на модела за генеративни цели
5. **Подобряване на модел**
6. **Взаимодействие с Llama чрез чат интерфейса на Gradio**

## 4.1. **Инсталиране** на библиотеки

- `Hugging Face Transformers`: Предоставя лесен начин за използване на предварително обучени модели.
- `PyTorch`: Нужен за операции за дълбоко обучение.
- `Accelerate`: Оптимизира PyTorch операциите, специално за графичен юзър интерфейс.

In [ ]:
pip install transformers torch accelerate

Инсталиране на най- новата версия на Gradio за да използваме `gr.ChatInterface()`

In [ ]:
!pip install --upgrade gradio

## 4.2. Предпоставки
1. Получаване на достъп до [Llama-2 7B](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf) в Hugging Face

2. Вход в Hugging Face и автентификация

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!huggingface-cli whoami

Cryptix37


## 4.3. Зареждане на модел и токенизатор
Зареждане на модела Llama-2 7B и неговия токенизатор.

Токенизаторът превръща въведения текст във формат, който моделът може да разбере и обработи.

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


## 4.4. Създаване на Llama канал
Настройване на канала за генериране на текст.

Той опростява процеса на подаване на входни данни към модела и получаването на генериран текст като изход.

In [ ]:
from transformers import pipeline

llama_pipeline = pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]



## 4.5. Подобряване на модела:
**Въвеждане на история за разговора:** Чрез промяна на структурата на Llama2 prompts:

```
<s>[INST] <<SYS>>
{{ system_prompt }}
<</SYS>>
{{ user_message }} [/INST]
```

## Improved Prompts:

### How to solve the following concerns:
1. **Lack of Conversation History**: A basic approach does not account for past interactions, making it less effective for maintaining a coherent conversation. We can mitigate this by using the right structure for Llama2 prompts:
```
<s>[INST] <<SYS>>
{{ system_prompt }}
<</SYS>>
{{ user_message }} [/INST]
```

2. **Non-Limited Customization**: Allow for advanced prompt formatting or handling system-level instructions.

3. **Must be ready for UI Integration**: Easy integration with user interface libraries like Gradio.



Параметри:
- `message` е съобщението, което изпращаме
- `history`е списък от наредени n-орки, които съдържат нашия разговор `[(user_msg1, bot_msg1), (usr_msg2, bot_msg2), ...]`

In [ ]:
SYSTEM_PROMPT = """<s>[INST] <<SYS>>
You are a helpful bot. Your answers are clear and concise.
<</SYS>>
"""

# Formatting function for message and history
def format_message(message: str, history: list, memory_limit: int = 3) -> str:
    """
    Formats the message and history for the Llama model.

    Parameters:
        message (str): Current message to send.
        history (list): Past conversation history.
        memory_limit (int): Limit on how many past interactions to consider.

    Returns:
        str: Formatted message string
    """
    # always keep len(history) <= memory_limit
    if len(history) > memory_limit:
        history = history[-memory_limit:]

    if len(history) == 0:
        return SYSTEM_PROMPT + f"{message} [/INST]"

    formatted_message = SYSTEM_PROMPT + f"{history[0][0]} [/INST] {history[0][1]} </s>"

    # Handle conversation history
    for user_msg, model_answer in history[1:]:
        formatted_message += f"<s>[INST] {user_msg} [/INST] {model_answer} </s>"

    # Handle the current message
    formatted_message += f"<s>[INST] {message} [/INST]"

    return formatted_message

## 4.6. Получаване на отговори
Чрез чат интерфейса на Gradio

In [ ]:
# Generate a response from the Llama model
def get_llama_response(message: str, history: list) -> str:
    """
    Generates a conversational response from the Llama model.

    Parameters:
        message (str): User's input message.
        history (list): Past conversation history.

    Returns:
        str: Generated response from the Llama model.
    """
    query = format_message(message, history)
    response = ""

    sequences = llama_pipeline(
        query,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=1024,
    )

    generated_text = sequences[0]['generated_text']
    response = generated_text[len(query):]  # Remove the prompt from the output

    print("Chatbot:", response.strip())
    return response.strip()


In [ ]:
import gradio as gr

gr.ChatInterface(get_llama_response).launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://491e95d76e79b4a343.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
